## Calculate student basket and course load

This notebook calculate weekly/daily student basket size (number of courses students are enrolled in), and the course load and credit hours of their basket.

In [3]:
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta
import time
from datetime import datetime, timedelta

df = pd.read_csv(TRANSACTION_DATA)
semester_dict = {'2012 Fall': 2128,'2013 Spring': 2132,'2013 Summer': 2135,'2013 Fall': 2138,'2014 Spring': 2142,'2014 Summer': 2145,'2014 Fall': 2148,'2015 Spring': 2152,'2015 Summer': 2155,'2015 Fall': 2158,'2016 Spring': 2162,'2016 Summer': 2165,'2016 Fall': 2168,'2017 Spring': 2172,'2017 Summer': 2175,'2017 Fall': 2178,'2018 Spring': 2182,'2018 Summer': 2185,'2018 Fall': 2188,'2019 Spring': 2192,'2019 Summer': 2195,'2019 Fall': 2198,'2020 Spring': 2202,'2020 Summer': 2205,'2020 Fall': 2208,'2021 Spring': 2212,'2021 Summer': 2215,'2021 Fall': 2218,'2022 Spring': 2222,'2022 Summer': 2225,'2022 Fall': 2228}
semester_dict = {v: k for k, v in semester_dict.items()}
df['semester_clean'] = df.semester_year_term_cd.map(semester_dict)
df['course_clean'] = df.subject_desc.map(str)  + ' ' + df['course_number'].map(str) 

## filter data to only contain actions that affect student enrollment status & are initiated by students

df_filtered = df[ (df['action_affects_enrollment_status_flag']=='Y') & (df['enrollment_intitiator_type']=='Student')]


start_phase_1 = {   '2016 Fall':'2016/04/18',
                    '2017 Spring':'2016/10/17',
                    '2017 Fall':'2017/04/17',
                    '2018 Spring':'2017/10/16',
                    '2018 Fall':'2018/04/18',
                    '2019 Spring':'2018/10/17',
                    '2019 Fall':'2019/04/15',
                    '2020 Spring':'2019/10/17',
                    '2020 Fall':'2020/04/13',
                    '2021 Spring':'2020/10/12',
                    '2021 Fall':'2021/04/26',
                    '2022 Spring':'2021/10/11'}

add_drop_cal = pd.read_excel('add drop calendar fa16-fa22 (1) (1).xlsx')


In [ ]:


def partition_days(start_date, end_date):
    '''
    Given 2 dates, partition the days between them into days
    '''

    days = {}
    current_date = start_date
    day_number = 1

    while current_date <= end_date:
        days[current_date.strftime('%Y-%m-%d')] = day_number
        current_date += timedelta(days=1)
        day_number += 1

    return days



### create a dictionary of partitioned days 
parts_dict = {}
for s in add_drop_cal['semester']:
    try:
        relevant_row = add_drop_cal[add_drop_cal['semester']==s]

        start = pd.to_datetime(start_phase_1[s])
        end = list(relevant_row['Change grading options'])[0]
        week_partitions = partition_days(start, end)
        parts_dict[s]=week_partitions
    except:
        pass
    
last_day_sem = dict(zip(add_drop_cal['semester'], add_drop_cal['Change grading options']))

def output_day_based_on_date(sem,input_date):
    
    sem_day_partitions = parts_dict[sem]
    try:
        return sem_day_partitions[input_date]
    except:
        pass


### load in data on course load & credit hours

In [ ]:
    
credit_hrs = df_filtered[['semester_clean', 'course_clean','units']].drop_duplicates()

credit_hrs_dict = {}

for index, row in credit_hrs.iterrows():
    semester = row['semester_clean']
    course = row['course_clean']
    units = row['units']
    
    if semester not in credit_hrs_dict:
        credit_hrs_dict[semester] = {}
    
    credit_hrs_dict[semester][course] = units

    


In [ ]:
fa_17_course_load = pd.read_csv(COURSELOAD_DATA_FA17)
sp_17_course_load = pd.read_csv(COURSELOAD_DATA_SP17)
fa_18_course_load = pd.read_csv(COURSELOAD_DATA_FA18)
sp_18_course_load = pd.read_csv(COURSELOAD_DATA_SP18)
fa_19_course_load = pd.read_csv(COURSELOAD_DATA_FA19)
sp_19_course_load = pd.read_csv(COURSELOAD_DATA_SP19)
fa_20_course_load = pd.read_csv(COURSELOAD_DATA_FA20)
sp_20_course_load = pd.read_csv(COURSELOAD_DATA_SP20)
sp_21_course_load = pd.read_csv(COURSELOAD_DATA_SP21)

course_load_df_dict={'2017 Fall': fa_17_course_load,
                    '2017 Spring': sp_17_course_load,
                     '2018 Fall': fa_18_course_load,
                     '2018 Spring': sp_18_course_load,
                    '2019 Fall': fa_19_course_load,
                     '2019 Spring': sp_19_course_load,
                     '2020 Fall': fa_20_course_load,
                     '2020 Spring': sp_20_course_load,
                     '2021 Spring': sp_21_course_load,
                                     
                    }

course_load_concat = pd.concat(course_load_df_dict).reset_index()
course_load_concat = course_load_concat[['level_0','course_name_number', 'tl1', 'me', 'ps',
       'cl_combined']]
course_load_concat.columns=['semester_clean', 'course_clean','time_load','mental_effort','psych_stress', 'cl_combined']


## create a dictionary of courseload 
course_load_dict = {}

for index, row in course_load_concat.iterrows():
    semester = row['semester_clean']
    course = row['course_clean']
    data_dict = {
        'time_load': row['time_load'],
        'mental_effort': row['mental_effort'],
        'psych_stress': row['psych_stress'],
        'cl_combined': row['cl_combined']
    }
    
    if semester not in course_load_dict:
        course_load_dict[semester] = {}
    
    if course not in course_load_dict[semester]:
        course_load_dict[semester][course] = []
    
    course_load_dict[semester][course].append(data_dict)

    


### calculate basket size & course load

In [ ]:
import tqdm
def calculate_courses_in_basket_real_time(df, course_filter_df = None ):
    '''
    calculates the number of courses in student basket for each student in a specific semester
    
    input:
        df = dataframe of transaction data for a specific semester
    
    output:
        student basket result df = df with student_id, semester_clean, basket_size,timeload, mental effort, 
        psych stress, course load, credit hours, enrollment_request_tmsp
    
    '''
    df.sort_values(by=['student_id', 'enrollment_request_tmsp'], inplace=True)
    if course_filter_df is not None:
        df = df.merge(course_filter_df, how = 'right', on = ['semester_clean', 'course_clean'])
    
    student_basket_dfs = []
    sem = df['semester_clean'].unique()[0]
    max_day = parts_dict[sem][str(last_day_sem[sem])[:10]]

    for student_id, student_df in tqdm.tqdm(df.groupby('student_id'), desc="Processing students"):
        semester = student_df['semester_clean'].iloc[0]
        student_basket = set()
        basket_size_changes = []

        for index, transaction in student_df.iterrows():
            course = transaction['course_clean']
            action = transaction['student_enrollment_status_outcome_cd']
            timestamp = transaction['enrollment_request_tmsp']
            day_num = transaction['day_num']

            if action == 'E':
                student_basket.add(course)
            elif action == 'D':
                student_basket.discard(course)
            
            tl_sum, me_sum, ps_sum, cl_sum, cr_sum = 0,0,0,0,0
            for cc in student_basket:
                try:
                    tl_sum+= course_load_dict[sem][cc][0]['time_load']
                    me_sum+= course_load_dict[sem][cc][0]['mental_effort']
                    ps_sum+= course_load_dict[sem][cc][0]['psych_stress']
                    cl_sum+= course_load_dict[sem][cc][0]['cl_combined']
                    cr_sum+= credit_hrs_dict[sem][cc]
                except:
                    pass

            basket_size_changes.append({
                'student_id': student_id,
                'semester_clean': semester,
#                 'enrollment_request_tmsp': timestamp,
                'day_num': day_num,
                'basket_size': len(student_basket),
                'tl':tl_sum,
                'me':me_sum,
                'ps':ps_sum,
                'cl':cl_sum,
                'ch':cr_sum
            })
        
        stu_basket_df = pd.DataFrame(basket_size_changes)
        try:
            ## since we only note down changes in student basket, there are missing basket
            ## calculation for some days, fill in missing days
            stu_basket_df = fill_missing_days(stu_basket_df, max_day)
        except:
            pass
        
        student_basket_dfs.append(stu_basket_df)

    return pd.concat(student_basket_dfs)



In [ ]:
import pandas as pd
import numpy as np
import tqdm

def fill_missing_days(student_df, max_day):
    '''
    Fill missing basket sizes for each student for every day of the semester by carrying over the previous day's basket size.
    '''
    filled_days = []
    prev_basket_size = None
    prev_tl, prev_me, prev_ps, prev_cl, prev_ch = None, None, None, None, None

    for day_num in range(1, int(max_day) + 1):
        day_data = student_df[student_df['day_num'] == day_num]
        if not day_data.empty:
            prev_basket_size = day_data['basket_size'].iloc[-1]
            prev_tl = day_data['tl'].iloc[-1]
            prev_me = day_data['me'].iloc[-1]
            prev_ps= day_data['ps'].iloc[-1]
            prev_cl= day_data['cl'].iloc[-1]
            prev_ch= day_data['ch'].iloc[-1]
            
        filled_days.append({
            'student_id': student_df['student_id'].iloc[0],
            'semester_clean': student_df['semester_clean'].iloc[0],
            'day_num': day_num,
            'basket_size': prev_basket_size,
            'tl':prev_tl,
            'me':prev_me,
            'ps':prev_ps,
            'cl':prev_cl,
            'ch':prev_ch
        })

    return pd.DataFrame(filled_days).fillna(0)


In [ ]:
### calculate for a specific semester
sems = df_filtered['semester_clean'].unique()
sems = [x for x in sems if 'Summer' not in x][1:]
s = '2018 Spring'
sem_df = df_filtered[df_filtered['semester_clean']==s]
sem_df['timestamp_date'] = sem_df['enrollment_request_tmsp'].str[:10]
sem_df['day_num'] = sem_df.apply(lambda x: output_day_based_on_date(x['semester_clean'],x['timestamp_date']),axis=1)
basket_sem_df = calculate_courses_in_basket_real_time(sem_df)
basket_sem_df.to_csv( s+'_student_daily_basket_cla_ch.csv')